Wczytanie Danych

In [88]:
import pandas as pd

df = pd.read_csv('Dane_mpsi.csv',sep='\t', encoding='utf-8')

Najprostsza predykcja zgonu

In [89]:
zgon_df = df.drop(columns=['KG', 'follow up 30 dni'])
zgon_df

,zgon,male sex,WIEK,Operowany przed przyjęciem (0/1),Interleukina 6,Prokalcytonina,MAP 1sza doba,pao2/fio2 1sza doba,Waga (kg),Wzrost (cm),BMI,Glukoza (1. gaz. 1sza doba),Lac (1. gaz. 1sza doba),BE (1. gaz. 1sza doba),SOFA - punktacja,Sepsa (0/1)
0,0,0,51,0,110,"6,49",80,"452,5",70,165,"25,71166208",32,"2,3","-23,4",5,1
1,0,1,72,1,0,"0,06","63,33333333","178,8888889",100,180,"30,86419753","8,6","2,7","-4,6",10,0
2,0,1,62,1,"111,5","7,04",120,1430,80,175,"26,12244898","7,2","2,4","-4,6",10,0
3,0,0,71,1,"51,7","0,51",70,"312,5",80,170,"27,6816609","6,5","0,6","-7,9",13,0
4,1,0,64,0,151844,"17,9",60,0,80,160,"31,25","4,2",18,"-19,5",0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
454,0,0,65,1,0,"0,02","83,33333333",408,75,165,"27,54820937","9,4",2,"22,1",7,0
455,0,1,62,1,"73,8","5,89",60,385,80,178,"25,2493372",11,"7,4","-15,3",13,1
456,1,0,73,1,50000,"43,04",90,"138,8",60,165,"22,03856749","12,8","10,5","-14,1",15,1
457,0,1,37,0,4823,"1,32",60,256,70,170,"24,22145329","5,6","14,6","-18,5",10,0


In [14]:
from sklearn.tree import DecisionTreeClassifier
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X_zgon = zgon_df.drop(columns=['zgon'])
y_zgon = zgon_df['zgon']

for col in X_zgon.select_dtypes(include='object').columns:
    X_zgon[col] = X_zgon[col].replace('Nie znaleziono', np.nan)
    X_zgon[col] = X_zgon[col].str.replace(',', '.').astype(float)
# Split the data
X_train_zgon, X_test_zgon, y_train_zgon, y_test_zgon = train_test_split(
    X_zgon, y_zgon, test_size=0.2, random_state=42, stratify=y_zgon
)

# Fit the model
clf_zgon = DecisionTreeClassifier(max_depth=4, random_state=42)
clf_zgon.fit(X_train_zgon, y_train_zgon)

# Evaluate on train
y_train_pred = clf_zgon.predict(X_train_zgon)
acc_train = accuracy_score(y_train_zgon, y_train_pred)
print(f"Train accuracy: {acc_train:.3f}")

# Evaluate on test
y_test_pred = clf_zgon.predict(X_test_zgon)
acc_test = accuracy_score(y_test_zgon, y_test_pred)
print(f"Test accuracy: {acc_test:.3f}")

Train accuracy: 0.763
Test accuracy: 0.674


In [27]:
rf_clf_zgon = RandomForestClassifier(random_state=42, max_depth=3, n_estimators=300)
rf_clf_zgon.fit(X_train_zgon, y_train_zgon)

# Evaluate on train
y_train_pred_rf_zgon = rf_clf_zgon.predict(X_train_zgon)
acc_train_rf_zgon = accuracy_score(y_train_zgon, y_train_pred_rf_zgon)
print(f"Random Forest Train Accuracy: {acc_train_rf_zgon:.3f}")

# Evaluate on test
y_test_pred_rf_zgon = rf_clf_zgon.predict(X_test_zgon)
acc_test_rf_zgon = accuracy_score(y_test_zgon, y_test_pred_rf_zgon)
print(f"Random Forest Test Accuracy: {acc_test_rf_zgon:.3f}")

# Feature importances
rf_importances_zgon = pd.Series(rf_clf_zgon.feature_importances_, index=X_train_zgon.columns)
print("Random Forest Feature importances:")
print(rf_importances_zgon.sort_values(ascending=False))

Random Forest Train Accuracy: 0.774
Random Forest Test Accuracy: 0.750
Random Forest Feature importances:
Lac (1. gaz. 1sza doba)             0.184812
BE (1. gaz. 1sza doba)              0.138252
SOFA - punktacja                    0.135183
WIEK                                0.127294
pao2/fio2 1sza doba                 0.063713
Interleukina 6                      0.057293
Glukoza (1. gaz. 1sza doba)         0.051647
BMI                                 0.051403
Prokalcytonina                      0.051377
MAP 1sza doba                       0.046236
Wzrost (cm)                         0.039931
Waga (kg)                           0.031442
Sepsa (0/1)                         0.011308
Operowany przed przyjęciem (0/1)    0.005076
male sex                            0.005036
dtype: float64


Okej, widzimy że bez feature engineringu mamy 0.66 accuracy i że najważniejsze cechy to Wiek, SOFA i Interleukina  

In [28]:
from itertools import combinations
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
import numpy as np
import pandas as pd

# Use already preprocessed X_zgon and y_zgon
numeric_cols = X_zgon.select_dtypes(include=[np.number]).columns.tolist()

feature_scores = []

# Generate all 2-feature combinations
for col1, col2 in combinations(numeric_cols, 2):
    # Sum
    new_feat_sum = X_zgon[col1] + X_zgon[col2]
    # Multiplication
    new_feat_mul = X_zgon[col1] * X_zgon[col2]
    # Difference
    new_feat_diff = X_zgon[col1] - X_zgon[col2]
    # Division (avoid division by zero)
    new_feat_div = X_zgon[col1] / (X_zgon[col2].replace(0, np.nan) + 1e-6)
    
    for op_name, feat in [
        (f"{col1}_plus_{col2}", new_feat_sum),
        (f"{col1}_mul_{col2}", new_feat_mul),
        (f"{col1}_minus_{col2}", new_feat_diff),
        (f"{col1}_div_{col2}", new_feat_div)
    ]:
        # Drop NaNs for this feature
        valid_idx = feat.notna()
        X_feat = feat[valid_idx].values.reshape(-1, 1)
        y_feat = y_zgon[valid_idx]
        if len(np.unique(X_feat)) < 2:
            continue  # skip constant features
        # Cross-validated accuracy
        clf = DecisionTreeClassifier(max_depth=3, random_state=42)
        score = cross_val_score(clf, X_feat, y_feat, cv=5, scoring='accuracy').mean()
        feature_scores.append((op_name, score))

# Sort and save top 10
top_10 = sorted(feature_scores, key=lambda x: x[1], reverse=True)[:10]
top_10_df = pd.DataFrame(top_10, columns=['feature', 'cv_accuracy'])
top_10_df.to_csv('top_10_combined_features.csv', index=False)
top_10_df

,feature,cv_accuracy
0,Lac (1. gaz. 1sza doba)_plus_SOFA - punktacja,0.679623
1,WIEK_div_Operowany przed przyjęciem (0/1),0.672626
2,WIEK_mul_Lac (1. gaz. 1sza doba),0.671166
3,Glukoza (1. gaz. 1sza doba)_div_Lac (1. gaz. 1...,0.666842
4,male sex_plus_Lac (1. gaz. 1sza doba),0.662375
5,Lac (1. gaz. 1sza doba)_mul_SOFA - punktacja,0.658003
6,BE (1. gaz. 1sza doba)_minus_SOFA - punktacja,0.655710
7,pao2/fio2 1sza doba_div_BE (1. gaz. 1sza doba),0.655160
8,WIEK_mul_BE (1. gaz. 1sza doba),0.653703
9,BE (1. gaz. 1sza doba)_minus_Sepsa (0/1),0.653703


In [ ]:
from sklearn.ensemble import RandomForestClassifier
results = []
for seed in range(10):
    rf_clf_feats = RandomForestClassifier(random_state=seed, max_depth=3, n_estimators=500)
    rf_clf_feats.fit(X_train_feats, y_train_zgon)
    y_test_pred_feats = rf_clf_feats.predict(X_test_feats)
    acc = accuracy_score(y_test_zgon, y_test_pred_feats)
    results.append(acc)

print(f"Max accuracy: {np.max(results):.3f}")
print(f"Min accuracy: {np.min(results):.3f}")
print(f"Average accuracy: {np.mean(results):.3f}")


Max accuracy: 0.761
Min accuracy: 0.685
Average accuracy: 0.719


In [53]:
from catboost import CatBoostClassifier

# Train CatBoost on base features
cat_clf = CatBoostClassifier(verbose=0, random_state=42, max_depth=2, n_estimators=500)
cat_clf.fit(X_train_zgon, y_train_zgon)

# Evaluate on train
y_train_pred_cat = cat_clf.predict(X_train_zgon)
acc_train_cat = accuracy_score(y_train_zgon, y_train_pred_cat)
print(f"CatBoost Train Accuracy: {acc_train_cat:.3f}")

# Evaluate on test
y_test_pred_cat = cat_clf.predict(X_test_zgon)
acc_test_cat = accuracy_score(y_test_zgon, y_test_pred_cat)
print(f"CatBoost Test Accuracy: {acc_test_cat:.3f}")

CatBoost Train Accuracy: 0.777
CatBoost Test Accuracy: 0.761


In [55]:
catboost_results = []
for seed in range(100):
    cat_clf_seed = CatBoostClassifier(verbose=0, random_state=seed, max_depth=2, n_estimators=500)
    cat_clf_seed.fit(X_train_zgon, y_train_zgon)
    y_test_pred_cat_seed = cat_clf_seed.predict(X_test_zgon)
    acc_cat = accuracy_score(y_test_zgon, y_test_pred_cat_seed)
    catboost_results.append(acc_cat)

print(f"CatBoost Max accuracy: {np.max(catboost_results):.3f}")
print(f"CatBoost Min accuracy: {np.min(catboost_results):.3f}")
print(f"CatBoost Average accuracy: {np.mean(catboost_results):.3f}")

CatBoost Max accuracy: 0.783
CatBoost Min accuracy: 0.717
CatBoost Average accuracy: 0.747
